# Numerical computation

1. Representation of integers in Python and NumPy
   - Maximum/minimum values, overflow
1. Concepts of
   - Absolute error
   - Relative error
   - Precision and representation error
   - Accuracy
2. Representation of floating point numbers in Python and NumPy
   - Maximum/minimum values, machine precision, overflow and underflow
   - Representation error
   - Roundoff error
   - Commutative, associative and distributive properties
   - Comparison of floating point values
   - Cancellation error
   - Summation example
   - Quadratic equation example
   - Extended precision
1. Classic very detailed reference about floating point numbers https://www.itu.dk/~sestoft/bachelor/IEEE754_article.pdf



In [8]:
import numpy as np

## Integers

### Division operation

In Python3, integer division is not closed, unless use `//` operator.  I.e., if you use '/' to divide two integers the result is always a floating point number even if the result is whole number.

In [9]:
print("Python3:    4/2", 4/2)
print("Python3:    3/2 =", 3/2, "    3//2 =", 3//2, "   1//2 =",1//2)

Python3:    4/2 2.0
Python3:    3/2 = 1.5     3//2 = 1    1//2 = 0


*NumPy behaves the same way*

In [10]:
a = np.arange(10)
print(a.dtype)
print(a)
print(a/2)
print(a/np.full(10,2))

int32
[0 1 2 3 4 5 6 7 8 9]
[0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5]
[0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5]


### Allowed values of integers

Native Python3 integers can be arbitrarily large (positive or negative)

In [11]:
i = 1
print(" m       2**m             2**m in binary format")
print("--    ----------     -------------------------------------")

for iteration in range(35):
    print("{0:2d}  {1:12d}  {1:40b}".format(iteration,i))
    i*=2

 m       2**m             2**m in binary format
--    ----------     -------------------------------------
 0             1                                         1
 1             2                                        10
 2             4                                       100
 3             8                                      1000
 4            16                                     10000
 5            32                                    100000
 6            64                                   1000000
 7           128                                  10000000
 8           256                                 100000000
 9           512                                1000000000
10          1024                               10000000000
11          2048                              100000000000
12          4096                             1000000000000
13          8192                            10000000000000
14         16384                           100000000000000
15      

However, NumPy has integers with a fixed size for compact storage and efficient computing (since this is what the computer hardware actually uses)

E.g., a 32-bit integer is a sequence of 32 bits (4 bytes) using two's complement representation (https://en.wikipedia.org/wiki/Two%27s_complement)

```
 [bit31, bit30, bit29, bit28, ..., bit2, bit1, bit0]
 ```

$i = - b_{31}*2^{31} +  b_{30}*2^{30} +  b_{29}*2^{29}+  b_{28}*2^{28} + \cdots + b_{2}*2^{2} + b_{1}*2^{1} + b_{0}*2^0 $

Most modern computers support in hardware 8-bit, 16-bit, 32-bit, and 64-bit integers (with and without signs)

The default integer in NumPy is 64-bit.

If you are curious, or if you ever need to do it by hand:
* Computing the binary representation of a positive number is straightforward.  In decreasing order, subtract powers of two less than the value.  For example, `265`.  The largest power of 2 less than 265 is $2^8=256$.  Subtracting 256 leaves $9$, which you can see is `8+1 = 2^3 + 2^0`, so the 32-bit binary representation of `265` is `00000000000000000000000100001001`.
* For a negative integer $-i$ the lower 31 bits are computed from $2^{31}-i$ and the 32nd bit is set.  So armed with the binary representation of +265 we can compute the two's-complement binary representation of -265 by first computing (in binary) $2^{31}-265$.

```
  10000000000000000000000000000000
- 00000000000000000000000010001001
= 01111111111111111111111101110111
```

and then seting the upper bit to obtain `11111111111111111111111011110111`.

**Exercise:** In two's-complement 32-bit format, what are the binary representations of 
* 3
* -3

If you want to check your answer look [here](https://www.exploringbinary.com/twos-complement-converter)


In [12]:
# Aside: Python can print binary representations of integers but does not internally use the
# two's-complement representation and so negative numbers appear just with a minus sign
print("{:b}".format(3))
print("{:b}".format(-3))

11
-11


Here's a function to convert an integer into a string containing its binary 32-bit two's-complement representation

In [13]:
def b(i, nbits=32):
    ''' Returns bit-string representation of a 32-bit two's-complement integer '''
    if i<-2**(nbits-1) or i>=2**(nbits-1):
        raise(ValueError)
    s = "0"
    if i<0:
        s = "1"
        i += 2**(nbits-1)
    for q in range(nbits-2,-1,-1):
        s += ["0","1"][(i&(1<<q))>>q]
    return s

print(-2147483648, b(-2147483648)) # The most negative integer

-2147483648 10000000000000000000000000000000


In [14]:
# Test it on some integers from numpy
import numpy as np
i = np.array([3],np.int32)
print("{0:11d} {1}".format(i[0],b(i[0])))
i[0] = -3
print("{0:11d} {1}".format(i[0],b(i[0])))


          3 00000000000000000000000000000011
         -3 11111111111111111111111111111101


The fixed width of NumPy integers means they cannot be arbitrarily large. The largest positive value you can put in a 32-bit signed integer must fit into 31 bits which is

$2^{31}-1$ = 2147483647

And the largest magnitude negative value is 
$-2^{31}$ = -2147483648

For the default 64-bit integers the limits are $2^{63}-1$ = 9223372036854775807 and $-2^{63}$=-9223372036854775808

**Exercise:** Explain the following behaviors

In [15]:
# Unexpected things can happen when you "overflow" a fixed-size integer

i = np.array([2**31-1],np.int32)
print("{0:11d} {1}".format(i[0],b(i[0])))
i += 1
print("{0:11d} {1}".format(i[0],b(i[0])))
print()
i = np.array([-2**31],np.int32)
print("{0:11d} {1}".format(i[0],b(i[0])))
i -= 1
print("{0:11d} {1}".format(i[0],b(i[0])))
print("{0:11d} {1}".format(-1,b(-1)))


 2147483647 01111111111111111111111111111111
-2147483648 10000000000000000000000000000000

-2147483648 10000000000000000000000000000000
 2147483647 01111111111111111111111111111111
         -1 11111111111111111111111111111111


In [16]:
# Explore increasing powers of 2
i = np.array([1],np.int32)
for iteration in range(40):
    print("{0:2d}  {1:11d}   {2}".format(iteration,i[0],b(i[0])))
    i *= 2

 0            1   00000000000000000000000000000001
 1            2   00000000000000000000000000000010
 2            4   00000000000000000000000000000100
 3            8   00000000000000000000000000001000
 4           16   00000000000000000000000000010000
 5           32   00000000000000000000000000100000
 6           64   00000000000000000000000001000000
 7          128   00000000000000000000000010000000
 8          256   00000000000000000000000100000000
 9          512   00000000000000000000001000000000
10         1024   00000000000000000000010000000000
11         2048   00000000000000000000100000000000
12         4096   00000000000000000001000000000000
13         8192   00000000000000000010000000000000
14        16384   00000000000000000100000000000000
15        32768   00000000000000001000000000000000
16        65536   00000000000000010000000000000000
17       131072   00000000000000100000000000000000
18       262144   00000000000001000000000000000000
19       524288   0000000000001

**Summary:** *these issues only affect very large integers in NumPy (or other programming languages including Python2) and most of the time things will behave as you expect.*

## Absolute error, relative error, significant digits, precision, and accuracy

These are central concepts to numerical computation

**Absolute error** is the magnitude of the error between an approximate result and the exact one $\epsilon_{\mathrm{abs}} = |x_\mathrm{exact} - x_{\mathrm{approx}}|$


In [17]:
x = (1.0/(1.0/7.0**0.5)**3)**(2/3) #  exact answer is 7
xexact = 7
print(x)
abserr = abs(x-7)
print(abserr)

7.000000000000001
8.881784197001252e-16


While absolute error is very important, it requires that we have some understanding about how big an error is bad.  

For instance, an error of a million would be a lot if counting the population of a town, but would likely be regarded as small if measuring the distance to the sun in meters (149,597,870,700m).

**Relative error** can be defined as ratio of the absolute error to the exact result 

$$ \epsilon_{\mathrm{rel}} = |x_\mathrm{exact} - x_{\mathrm{approx}}| / |x_\mathrm{exact}| $$

but other definitions are also useful depending on the circumstance, e.g.,

$$ \epsilon_{\mathrm{rel}} = |x_\mathrm{exact} - x_{\mathrm{approx}}| / \mathrm{max}(|x_\mathrm{exact}|,|x_\mathrm{approx}|) $$

$$ \epsilon_{\mathrm{rel}} = |x_\mathrm{exact} - x_{\mathrm{approx}}| / |x_\mathrm{approx}| $$

From the perspective of relative error we can see the above computation was quite accurate

In [18]:
relerr = abs(x-xexact)/xexact
print(relerr)

1.2688263138573217e-16


**Significant digits:** Relative error can also be interpreted as the number of significant figures or digits ($N$) in the value, where $N$ is computed as the largest integer such that 

$$ \epsilon_{\mathrm{rel}} < 0.5 * 10^{-N} $$

or 

$$
N \approx \mathrm{floor}( -\log_{10} (2* \epsilon_{\mathrm{rel}}) ),
$$

assuming that $\epsilon_{\mathrm{rel}}> 0$.


In [19]:
from math import *
sigfig = int(-log10(2*relerr))
print(sigfig)

15



**Precision** is often stated as the number of digits (or relative error) used to store or compute a result. Sometimes this might be stated as relative error, or the absolute error (e.g., if fixed-point arithmetic is being used instead of floating point).

For example, here is an approximation to $\pi$
```
   piapprox = 3.1415243098283216
```
that has been specified with a precision of 17 digits.  We will see below that IEEE double-precision floating-point arithmetic has a precision of about 16 digits (and specifying a correctly rounded value can require a few more digits).

Note: if we had specified more digits that they would have been discarded --- double-precision simply cannot hold any more information

In [20]:
piapprox = 3.1415243098283216
print(piapprox)
piapprox = 3.1415243098283216217439821748972198
print(piapprox)

3.1415243098283216
3.1415243098283216


**Representation error:** because of finite precision some values cannot be exactly represented --- this is usually not a problem but can lead to some unexpected outputs.

In [21]:
print(3.1415243098283217)  # focus on the last digit
print("%.17f" % 0.2)       # focus on the last digit --- running error

3.1415243098283216
0.20000000000000001


In [22]:
import math
math.sin(math.pi)          # pi is not exactly representable, so sin(pi) cannot be exactly zero

1.2246467991473532e-16

**Accuracy** is often stated as the number of signficant figures in a value comparing to the exact or true value. Sometimes relative or absolute error might be used.

Clearly the attainable accuracy is limited by the precision of computation, but there may be other limits
*  finite accuracy of values input into a calculation
*  finite accuracy of an algorithm to compute something

Looking at the above approximate value for $\pi$ that was stated with a precision of 17 digits we can see that it is only accurate to a little more than 4 decimal digits.

In [23]:
print("approx:", piapprox)
print(" exact:", math.pi, "(i.e., the closest floating-point representation of pi)")
pirelerr = abs(piapprox-math.pi)/math.pi
print("relerr:", pirelerr)
N = int(-log10(2*pirelerr))
print("sigfig:", N)

approx: 3.1415243098283216
 exact: 3.141592653589793 (i.e., the closest floating-point representation of pi)
relerr: 2.1754494935355024e-05
sigfig: 4


## Floating point numbers 

Nearly all languages, including Python and NumPy, use the computer hardware suported IEEE 754 representation. 
* NumPy supports the most commonly used single precision (32-bit) and double precision (64-bit) floating-point numbers as `float32` and `float64`.
* NumPy also supports the newer half precision (16 bits) as `float16`.

E.g., in 64-bit (https://en.wikipedia.org/wiki/Double-precision_floating-point_format)

$$(-1)^s \times 1.m \times 2^{e-1023}$$

- 1 bit for sign ($s$)
- 11 bits for exponent ($e$)
  - In range -1024 to +1023 in binary, or $\pm$308 in decimal.  Thus, numbers larger than circa $10^{308}$ will overflow, and numbers smaller than circa $10^{-308}$ will underflow (gradually due to denormalized representation)
- 52 bits for mantissa ($m$) giving 53 bits of signifance 
  - 53 bits since we know for a non-zero number that the leading binary digit is 1 so we don't bother storing that
  - $2^{-53} \approx$ `1.11e-16`
- Special values are reserved to represent
  - Signed zero
  - Overflow (number too in magnitude to represent) --- $\pm \infty$
  - Not a number (result is not a valid number) --- `NaN`

For native Python these limits and associated values can be found in `sys.float_info`

In [24]:
import sys
print(sys.float_info)
print()
print("maximum floating point number is", sys.float_info.max)

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

maximum floating point number is 1.7976931348623157e+308


In [25]:
print(np.finfo(np.float))

Machine parameters for float64
---------------------------------------------------------------
precision =  15   resolution = 1.0000000000000001e-15
machep =    -52   eps =        2.2204460492503131e-16
negep =     -53   epsneg =     1.1102230246251565e-16
minexp =  -1022   tiny =       2.2250738585072014e-308
maxexp =   1024   max =        1.7976931348623157e+308
nexp =       11   min =        -max
---------------------------------------------------------------



C:\Users\user\AppData\Local\Temp/ipykernel_29548/891928311.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  print(np.finfo(np.float))


In [26]:
# Illustrating what happens when you exceed the most postitive exponent --- overflow
x = 10.0**308
print(x)
x *= 2.0
print(x)

1e+308
inf


In [27]:
# Illustrating what happens when you exceed the most postitive exponent and mantissa --- overflow
import sys
x = sys.float_info.max
print(x)
x = x*1.0000000000000002
print(x)

1.7976931348623157e+308
inf


In [28]:
# Illustrating what happens as you approach and exceed the most negative exponent (i.e., very small numbers) --- gradual underflow
x = 1.23456789012345678e-307
while x>0:
    print(x)
    x *= 0.1
print(x)

1.2345678901234568e-307
1.2345678901234567e-308
1.234567890123457e-309
1.23456789012346e-310
1.2345678901233e-311
1.234567890124e-312
1.2345678901e-313
1.23456789e-314
1.23456789e-315
1.2345679e-316
1.234568e-317
1.234566e-318
1.23457e-319
1.2347e-320
1.235e-321
1.24e-322
1.5e-323
0.0


**Machine epsilon** (https://en.wikipedia.org/wiki/Machine_epsilon) is the smallest positive number such that 

$1 + \epsilon \ne 1$

It is the **relative error** for floating point computation.

For any real value $x$ (assuming no over/underflow) there exists a numerical representation $x^\prime$ such that $|x-x^\prime| < \epsilon |x|$

In [29]:
# Computing epsilon (normally would just look in sys.float_info)
for n in range(-60,1):
    epsilon = 2.0**n
    print(n, epsilon, 1.0+epsilon)
    if (1.0+epsilon) != 1.0:
        break
print()
print(sys.float_info.epsilon)

-60 8.673617379884035e-19 1.0
-59 1.734723475976807e-18 1.0
-58 3.469446951953614e-18 1.0
-57 6.938893903907228e-18 1.0
-56 1.3877787807814457e-17 1.0
-55 2.7755575615628914e-17 1.0
-54 5.551115123125783e-17 1.0
-53 1.1102230246251565e-16 1.0
-52 2.220446049250313e-16 1.0000000000000002

2.220446049250313e-16


**Rounding error:** Is related to representation error already introduced above. While some numbers and floating computations can be, or can appear to be, exact, most suffer rounding error because the significand has only 53 bits, and the last bit must usually be rounded. 

This gives you 15-16 significant decimal figures.

In IEEE 754 floating point arithmetic the default rounding mode is towards the closest exactly representable number.  Other rounding modes are available (e.g., round to zero, etc.) but are rarely needed.

In [30]:
import math
print(0.3/0.1-3.0)

-4.440892098500626e-16


The IEEE standard requires that the result of addition, subtraction, multiplication, division, square root, remainder, and conversion between integer and floating-point be *correctly rounded*.  It is not possible to do this efficiently for transcendental functions (e.g., `exp`) but these days most math libraries do correctly round all values and offer slightly less accurate modes (errors in the last 1-3 bits) that are potentially much faster.

More precisely, let $\times = +, -, *, /, \ldots$ in exact arithmetic and let $\otimes$ be the corresponding floating-point operation. Assuming no under/overflow, IEEE 754 arithmetic guarantees that given two floating-point values $x$ and $y$ that $|(x*y) - (x\otimes y)| < \epsilon |x*y|$. 

### Floating point arithmetic is *commutative* but *not associative* and *not distributive*

Commutative means $x \otimes y = y\otimes x$ and in floating point arithmetic this is true for $\otimes=+$ or $\otimes=*$.

Associative means $(x \otimes y) \otimes z = x \otimes (y \otimes z)$ where sub-expressions within parentheses are evaluated first.

Distributive means $x\otimes(y + z) = x\otimes y + x \otimes z$ for $\otimes = *, /$.

In [31]:
x=239480912809.2930841092
y=8309482109.193284092183018
z=1.328488213048321094
print(x*y == y*x)  # * commutes
print(x+y == y+x)  # + commutes
print(x-y == -(y-x)) # - commutes (taking care of sign)
print((x+y)+z == x+(y+z)) # + is NOT exactly associative in floating point
val1 = (x+y)+z
val2 = x+(y+z)
relerr = abs(val1-val2)/val1
print("relative error in associative test", relerr)
print(x*(y+z)==x*y+x*z) # * is NOT exactly distributive in floating point
val1 = x*(y+z)
val2 = x*y+x*z
relerr = abs(val1-val2)/val1
print("relative error in distributive test", relerr)

True
True
True
False
relative error in associative test 1.2315884211280873e-16
False
relative error in distributive test 1.3173314488952503e-16


**Reliably comparing floating-point values**

You must pay attention when comparing two floating point numbers --- since floating computation is imprecise, comparing two numbers should be done allowing for some reasonable error.  But what is reasonable depends on what accuracy you are expecting --- i.e., often you have to decide, but a reasonable default can be obtained from the machine epsilon assuming just rounding error is present.

The easiest way to do this is using `math.isclose` (https://docs.python.org/3/whatsnew/3.5.html#pep-485-a-function-for-testing-approximate-equality)

*Question:* Why does adding then subtracting 100 lose significance in the value of $\pi$?

In [32]:
x = (math.pi+100.0)-100.0
print(x==math.pi, x-math.pi)
print(math.isclose(x, math.pi, rel_tol=1e-14)) # *******************************

False 3.552713678800501e-15
True


**Cancellation error:** (loss of significance, https://en.wikipedia.org/wiki/Loss_of_significance) is a more pernicious problem.  

Adding/subtracting numbers of similar magnitude to obtain a relatively small result or intermediate value can lose significant digits, sometimes catastrophically.

In [34]:
x = 0.1234567890123456
y = 0.1234567890123
print(x)
print(y)
print(x-y)
print((1.0+2.0**52)-2.0**52)
print((1.0+2.0**53)-2.0**53)
print((math.pi+1e8)-1e8, math.pi) 
print((math.pi+1e8)-1e8 - math.pi) # subtract too close will have different float accuracy

0.1234567890123456
0.1234567890123
4.558853294867049e-14
1.0
0.0
3.141592651605606 3.141592653589793
-1.984187036896401e-09


**Exercise:** Summing data with varying magnitude and sign.

Below we first define a function to make a random value that has a large variation in magnitude but is always positive.  Each value is exactly represented as a floating point number since each is just a power of 2.

In [35]:
from random import random

def ranval():
    return 2.0**((random()-0.5)*100)

# Print a few out to see what they look like
for i in range(8):
    print(ranval())

4.891335790108483e-06
10.988759697774913
490936075.574317
1770824207126.2043
0.00010067982297399
0.06723608469531125
3.69988792321518e-05
3.116155173360896e-10


Let's sum a list of 1000 such values

In [36]:
values=[ranval() for i in range(1000)]
print("sum in original order:   ", sum(values))

sum in original order:    1.7683343012977808e+16


**Question:** Is there rounding error when computing the sum?

**Question:** Is cancellation error a concern here?

**Question:** Should the order of summation greatly affect the result?

In [37]:
values.reverse()
print("sum in reverse order:   ", sum(values))
print("sum in sorted order:    ", sum(sorted(values)))

sum in reverse order:    1.7683343012977806e+16
sum in sorted order:     1.7683343012977822e+16


We can see that any variation in the relative error is consistent with machine epsilon --- it is just rounding error.


Next we make a list of 2000 elements --- the first 1000 are random values computed with the first function and the second 1000 are just the negative of the first.

So we know the exact sum should be zero.

In [ ]:
values=[ranval() for i in range(1000)]
values = values + [-value for value in values]
print("sum in original order:   ", sum(values))


**Question:** Is cancellation error a concern here?

**Question:** Why should the order of summation matter?

**Question:** How can we get the computer to give us the "correct" answer?

**Question:** In general, if we don't know the "exact" result, what do we even mean by "correct?"

In [ ]:
print("sum in original order:  ", sum(values))
values.reverse()
print("sum in reverse order:   ", sum(values))
print("sum in sorted order:    ", sum(sorted(values)))
print("sum in abs sorted order:", sum(sorted(values,key=abs)))

In general even summing sorted data will still have some rounding error and may not even be the optimal algorithm --- we only get zero error here because of this special test case.

If you are concerned about the accuracy of a sum of floating point values, you could try using the `math.fsum` function.

In [ ]:
import math
math.fsum(values)

### Example of the quadratic equation

A classic example for numerical woes is the standard expression for the roots of a quadratic equation

$ a x^2 + b x + c = 0$

$ x = \frac{-b \pm \sqrt{b^2 - 4 a c}}{2 a}$

Three aspects of numerical computation conspire to make problems for this simple formula when $|ac| \ll b^2$:
*  Rounding error when computing $b^2 - 4 a c$
*  Cancellation error when computing $-b \pm \sqrt{b^2 - 4 a c}$ if $|4ac|\ll b^2$.

Consider adding two numbers $p+q$.  We've already seen that if $q$ is small compared to $p$ the addition operation must discard some of the digits in $q$. In the worst cast, if $|q|<\epsilon |p|$ then $p+q=p$ in floating-point arithmetic (where $\epsilon$ is the machine epsilon).


In [ ]:
p=1.0
q=1.2345678901234567e-12
print(p,q,p+q)

Coming back to the quadratic equation problem, imagine that $|ac| < \epsilon b^2$.
* The floating-point value for $b^2 - 4 a c$ will be computed as $b^2$ --- can you explain why?
* As a result (and assuming that $b>0$) we will compute $-b + \sqrt{b^2 - 4 a c}$ to have the value zero (give or take a bit of rounding error). This is the catastrophic cancellation error --- we have lost all information.
* Again assuming $|ac| \ll b^2$, a little bit of math (Taylor series) tells us that the correct answer is  $-b + \sqrt{b^2 - 4 a c} \approx + 2 ac/b$ and so that the corresponding root is $x \approx c/b$
* Similar problems arise for the other root if $b<0$.

These errors can be avoided by taking advantage of the alternative formula 

$$x = \frac{2c}{-b \mp \sqrt{b^2 - 4 a c}}$$

with '-' when $b\ge 0$ and '+' when $b<0$.

The original and alternative algorithms are implemented below.

In [38]:
def roots1(a,b,c, dtype=np.float):
    r = np.sqrt(b**dtype(2) - dtype(4)*a*c)
    return (-b+r)/(dtype(2)*a), (-b-r)/(dtype(2)*a)

def roots2(a,b,c, dtype=np.float):
    r = np.sqrt(b**dtype(2) - dtype(4)*a*c)
    return dtype(2)*c/(-b-r), dtype(2)*c/(-b+r)

#
dtype = np.float32
a, b, c = 0.05010, -98.78, 5.015

print("roots1:", roots1(dtype(a),dtype(b),dtype(c), dtype))
print("roots2:", roots2(dtype(a),dtype(b),dtype(c), dtype))

print("accurate: ", roots1(a,b,c, np.float))

roots1: (1971.6058, 0.05078649)
roots2: (1970.9927, 0.050770696)
accurate:  (1971.605915932872, 0.050770693874568965)


C:\Users\user\AppData\Local\Temp/ipykernel_29548/4197972976.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def roots1(a,b,c, dtype=np.float):
C:\Users\user\AppData\Local\Temp/ipykernel_29548/4197972976.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def roots2(a,b,c, dtype=np.float):
C:\Users\user\AppData\Local\Temp/ipykernel_295

We can combine the two formulas as follws:

In [39]:
def roots_hybrid(a,b,c, dtype=np.float):
    r = np.sqrt(b**dtype(2) - dtype(4)*a*c)
    x1 = (-b+r)/(dtype(2)*a) if b<0 else dtype(2)*c/(-b-r)
    x2 = (-b-r)/(dtype(2)*a) if b>0 else dtype(2)*c/(-b+r)
    return x1, x2

print("hybrid: ", roots_hybrid(a,b,c, np.float32))

hybrid:  (1971.605915932872, 0.050770693874611875)


C:\Users\user\AppData\Local\Temp/ipykernel_29548/542171486.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def roots_hybrid(a,b,c, dtype=np.float):


**Extended and arbitrary precision:** 
* NumPy supports the "extended precision" (typically 80 bit represented as 128 bits) `longfloat` (or `longdouble`). Older version of NumPy used to call `longdouble` as "`float128`", which was a misnomer and has been removed.
* Some libraries and packages (e.g., Maple or Mathematica) can provide greater (varialbe or arbitrary) precision at the cost of a loss of speed. For Python, the module `mpmath` (http://mpmath.org/) provides arbitrary precision arithmetic and we use this a bit further below.

With arbitrary-precision arithmetic, we can use more bits in the mantissa and have arbitrarily large exponents --- but the price is a significant loss of speed.  Also, it is not a magical solution, especially if there is only finite precision or accuracy in the input data.

In [40]:
a, b, c = 0.05010, -98.78, 5.015

dtype = np.longfloat
print("extended_precision roots1: ", roots1(dtype(a),dtype(b),dtype(c), dtype))
print("extended_precision roots2: ", roots2(dtype(a),dtype(b),dtype(c), dtype))
print("extended_precision hybrid: ", roots_hybrid(dtype(a),dtype(b),dtype(c), dtype))

import mpmath as mp
saveprec, mp.mp.prec = mp.mp.prec, 108 # set precision to 108 bits
dtype = mp.mpf

print("108-bit precision roots1: ", roots1(dtype(a),dtype(b),dtype(c), dtype))
print("108-bit precision roots2: ", roots2(dtype(a),dtype(b),dtype(c), dtype))
print("108-bit precision hybrid: ", roots_hybrid(dtype(a),dtype(b),dtype(c), dtype))

extended_precision roots1:  (1971.605915932872, 0.050770693874568965)
extended_precision roots2:  (1971.6059159345382, 0.050770693874611875)
extended_precision hybrid:  (1971.605915932872, 0.050770693874611875)
108-bit precision roots1:  (mpf('1971.6059159328719689255188750410376'), mpf('0.050770693874611872353338345755832257'))
108-bit precision roots2:  (mpf('1971.6059159328719689255188750179019'), mpf('0.050770693874611872353338345755236374'))
108-bit precision hybrid:  (mpf('1971.6059159328719689255188750410376'), mpf('0.050770693874611872353338345755236374'))
